## How is Topic Modeling and NLP useful?

- Imagine you have take all the songs that made the Top 100 billboard Charts since 1970. How could you then use topic modeling and NLP to create some  useful analysis?


- You work for a online news aggregator that takes news articles from all over the internet and sends out important articles to the users. The problem is each website has a different way of classifying their news stories. How could topic modeling and NLP to help you?


- You work for Ted Talks and want to predict the number of views each video will get.  Right now you only have the text of the speeches, their length, and the number of views. How could you use NLP and topic modelling to help with your predictive model?  

In [2]:
pip install gensim

    100% |████████████████████████████████| 24.7MB 1.1MB/s eta 0:00:01   17% |█████▋                          | 4.3MB 19.9MB/s eta 0:00:02    38% |████████████▎                   | 9.5MB 25.2MB/s eta 0:00:01    58% |██████████████████▊             | 14.4MB 26.9MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 10.8MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 25.3MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 2.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.4MB 2.9MB/s ta 0:00:011
  Stored in directory: /Users/justintennenbaum/Library/Caches/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
You are using pip version 19.2.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/justintennenbaum/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## The Data

In [4]:
import pandas as pd
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head(30)

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc
10000,From: a207706@moe.dseg.ti.com (Robert Loper)\n...,7,rec.autos
10001,From: kimman@magnus.acs.ohio-state.edu (Kim Ri...,6,misc.forsale
10002,From: kwilson@casbah.acns.nwu.edu (Kirtley Wil...,2,comp.os.ms-windows.misc
10003,Subject: Re: Don't more innocents die without ...,0,alt.atheism
10004,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,0,alt.atheism


## Model Explanation Resources 

https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

https://www.youtube.com/watch?v=DWJYZq_fQ2A

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24



<img src='https://www.machinelearningplus.com/wp-content/uploads/2018/03/Inferring-Topic-from-Keywords-1024x666.png' img/>

## Data Pre-processing

In [5]:
stemmer = SnowballStemmer('english')

In [6]:
WordNetLemmatizer().lemmatize(text, pos='v')

NameError: name 'text' is not defined

In [7]:
def lemmatize_stemming(text):
    word = WordNetLemmatizer().lemmatize(text, pos='v')
#     print('token',word)
    return stemmer.stem(word)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [8]:
df.iloc[0].values[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

### Select a document to preview after preprocessing.

In [9]:
doc_sample = df.iloc[0].values[0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['From:', 'lerxst@wam.umd.edu', "(where's", 'my', 'thing)\nSubject:', 'WHAT', 'car', 'is', 'this!?\nNntp-Posting-Host:', 'rac3.wam.umd.edu\nOrganization:', 'University', 'of', 'Maryland,', 'College', 'Park\nLines:', '15\n\n', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw\nthe', 'other', 'day.', 'It', 'was', 'a', '2-door', 'sports', 'car,', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/\nearly', '70s.', 'It', 'was', 'called', 'a', 'Bricklin.', 'The', 'doors', 'were', 'really', 'small.', 'In', 'addition,\nthe', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body.', 'This', 'is', '\nall', 'I', 'know.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name,', 'engine', 'specs,', 'years\nof', 'production,', 'where', 'this', 'car', 'is', 'made,', 'history,', 'or', 'whatever', 'info', 'you\nhave', 'on', 'this', 'funky', 'looking', 'car,', 'please', 'e-mail.\n\nThanks,\n-', 

In [10]:
processed_docs = df['content'].map(preprocess)


In [11]:
processed_docs[:20]

0        [lerxst, thing, subject, nntp, post, host, org...
1        [guykuo, carson, washington, subject, clock, p...
10       [irwin, cmptrc, lonestar, irwin, arnstein, sub...
100      [tchen, magnus, ohio, state, tsung, chen, subj...
1000     [dabl, lindbergh, subject, diamond, mous, curs...
10000    [dseg, robert, loper, subject, nntp, post, hos...
10001    [kimman, magnus, ohio, state, richard, subject...
10002    [kwilson, casbah, acn, kirtley, wilson, subjec...
10003    [subject, innoc, death, penalti, bobb, vice, r...
10004    [livesey, solntz, livesey, subject, genocid, c...
10005    [dsto, david, silver, subject, fractal, genera...
10006    [subject, mike, francesa, predict, gajarski, p...
10007    [netcom, netcom, eric, townsend, subject, insu...
10008    [cunixb, columbia, gari, dare, subject, covera...
10009    [sehari, iastat, babak, sehari, subject, disk,...
1001     [danmg, grok, columbiasc, daniel, adam, subjec...
10010    [henri, toronto, henri, spencer, subject, luna.

### Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 addit
1 bodi
2 bricklin
3 bring
4 bumper
5 call
6 colleg
7 door
8 earli
9 engin
10 enlighten


### Filter out tokens that appear in:

- less than 15 documents (absolute number) 
- more than 0.5 documents (fraction of total corpus size, not absolute number).
- after the above two steps, keep only the first 100000 most frequent tokens.

In [13]:
dictionary.filter_extremes(no_below=20, no_above=0.66, keep_n=10000)


In [14]:
len(dictionary)

5433

### Gensim doc2bow

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(11, 1),
 (20, 1),
 (22, 1),
 (27, 1),
 (31, 1),
 (138, 1),
 (141, 3),
 (237, 1),
 (242, 1),
 (268, 5),
 (395, 2),
 (580, 1),
 (585, 1),
 (614, 1),
 (641, 1),
 (685, 1),
 (793, 1),
 (814, 1),
 (855, 1),
 (897, 1),
 (960, 1),
 (1020, 1),
 (1341, 1),
 (1350, 1),
 (1510, 1),
 (1556, 1),
 (1619, 1),
 (1767, 2),
 (1823, 1),
 (1924, 1),
 (2383, 1),
 (2526, 1),
 (2568, 1),
 (2690, 1),
 (2788, 1),
 (4150, 1),
 (4326, 1),
 (4530, 2),
 (5013, 1)]

#### Preview Bag Of Words for our sample preprocessed document.

In [16]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 11 ("host") appears 1 time.
Word 20 ("nntp") appears 1 time.
Word 22 ("post") appears 1 time.
Word 27 ("spec") appears 1 time.
Word 31 ("univers") appears 1 time.
Word 138 ("state") appears 1 time.
Word 141 ("window") appears 3 time.
Word 237 ("write") appears 1 time.
Word 242 ("richard") appears 1 time.
Word 268 ("program") appears 5 time.
Word 395 ("silver") appears 2 time.
Word 580 ("secur") appears 1 time.
Word 585 ("true") appears 1 time.
Word 614 ("gate") appears 1 time.
Word 641 ("econom") appears 1 time.
Word 685 ("high") appears 1 time.
Word 793 ("support") appears 1 time.
Word 814 ("meet") appears 1 time.
Word 855 ("correct") appears 1 time.
Word 897 ("task") appears 1 time.
Word 960 ("current") appears 1 time.
Word 1020 ("major") appears 1 time.
Word 1341 ("oper") appears 1 time.
Word 1350 ("promis") appears 1 time.
Word 1510 ("dept") appears 1 time.
Word 1556 ("server") appears 1 time.
Word 1619 ("user") appears 1 time.
Word 1767 ("multi") appears 2 time.
Word 1823 ("e

### TF-IDF
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [17]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.16523076661355718),
 (1, 0.1677664905617056),
 (2, 0.15012097927517018),
 (3, 0.2856620819895799),
 (4, 0.11433072900048068),
 (5, 0.15159954267776288),
 (6, 0.38912221066291464),
 (7, 0.16881835532183245),
 (8, 0.12272982017825479),
 (9, 0.2633178454540476),
 (10, 0.16437312316877342),
 (11, 0.041885373029623627),
 (12, 0.13936358719351144),
 (13, 0.05320037340398117),
 (14, 0.17831230414471078),
 (15, 0.1613726005913341),
 (16, 0.1017696733142144),
 (17, 0.23327140545553357),
 (18, 0.16211755838097908),
 (19, 0.2803124591711666),
 (20, 0.042624920597482026),
 (21, 0.1848445683855343),
 (22, 0.03254024351960623),
 (23, 0.14859699959830053),
 (24, 0.15962827481075148),
 (25, 0.1814706209577019),
 (26, 0.14469438092060538),
 (27, 0.2096099743383186),
 (28, 0.1928032035466336),
 (29, 0.08427773412644116),
 (30, 0.08372804371108514),
 (31, 0.04439481147976632),
 (32, 0.13714379483744416),
 (33, 0.0841337463198108)]


## Running LDA using Bag of Words

Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [20]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=4, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"file" + 0.016*"entri" + 0.010*"list" + 0.010*"send" + 0.009*"program" + 0.009*"output" + 0.008*"section" + 0.007*"request" + 0.007*"period" + 0.007*"firearm"
Topic: 1 
Words: 0.018*"encrypt" + 0.015*"chip" + 0.012*"clipper" + 0.011*"secur" + 0.009*"govern" + 0.007*"key" + 0.007*"escrow" + 0.006*"inform" + 0.006*"phone" + 0.006*"write"
Topic: 2 
Words: 0.010*"write" + 0.009*"post" + 0.009*"articl" + 0.008*"know" + 0.007*"good" + 0.007*"drug" + 0.006*"help" + 0.006*"univers" + 0.006*"like" + 0.006*"time"
Topic: 3 
Words: 0.016*"write" + 0.015*"articl" + 0.011*"univers" + 0.010*"post" + 0.008*"uiuc" + 0.008*"year" + 0.008*"host" + 0.007*"nntp" + 0.007*"bike" + 0.006*"know"
Topic: 4 
Words: 0.021*"space" + 0.016*"nasa" + 0.008*"orbit" + 0.006*"launch" + 0.005*"year" + 0.005*"center" + 0.005*"research" + 0.005*"data" + 0.005*"imag" + 0.005*"satellit"
Topic: 5 
Words: 0.011*"think" + 0.010*"write" + 0.010*"exist" + 0.009*"moral" + 0.009*"peopl" + 0.008*"christian" + 0

### Running LDA using TF-IDF


In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

### Performance evaluation by classifying sample document using LDA Bag of Words model
We will check where our test document would be classified.

In [ ]:
processed_docs[4310]

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

### Performance evaluation by classifying sample document using LDA TF-IDF model.

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

### Testing model on unseen document


In [ ]:
unseen_document = ''
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("\n Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

## Compute Model Perplexity and Coherence Score
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is.  Topic coherence score, in particular, has been more helpful.

In [23]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.769944469800711

Coherence Score:  0.43388944506097127


https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
    https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

## Doing this with sklearn

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0